In [ ]:
from sqlalchemy.engine import create_engine
from sqlalchemy import inspect
from sqlalchemy import text
from pprint import pprint

import csv
import glob


mmCourse = 'engy'

sqlite_engine = create_engine('sqlite:////Users/saisun/ProjectCode/uqx_tools/micromastersreports/databases/' + mmCourse + '_verified.db')
sqlite_connection = sqlite_engine.connect()

grade_file_path = '/Users/saisun/ProjectCode/uqx_tools/micromastersreports/data/' + mmCourse + '/'
grade_file_postfix = "_grade_report_*.csv"
profile_file_postfix = "_student_profile_info_*.csv"

mmCoursesMeta = {
    'buslead': {
        "buslead1x": {"title": "UQx_BUSLEAD1x",  'grade': 0.65},
        "buslead2x": {"title": "UQx_BUSLEAD2x",  'grade': 0.65},
        "buslead3x": {"title": "UQx_BUSLEAD3x",  'grade': 0.65},
        "buslead4x": {"title": "UQx_BUSLEAD4x",  'grade': 0.65},
        "buslead5x": {"title": "UQx_BUSLEAD5x",  'grade': 0.65},
    },
    'corpinn': {
        "corpinn1x": {"title": "UQx_CORPINN1x", 'grade': 0.7},
        "corpinn2x": {"title": "UQx_CORPINN2x",  'grade': 0.7},
        "corpinn3x": {"title": "UQx_CORPINN3x",  'grade': 0.7},
        "corpinn4x": {"title": "UQx_CORPINN4x",  'grade': 0.7},
        "corpinn5x": {"title": "UQx_CORPINN5x",  'grade': 0.5},
    },
    'engy': {
        "engy0x": {"title": "UQx_ENGY0x", 'grade': 0.7},
        "engy1x": {"title": "UQx_ENGY1x", 'grade': 0.7},
        "engy2x": {"title": "UQx_ENGY2x", 'grade': 0.7},
        "engy3x": {"title": "UQx_ENGY3x", 'grade': 0.7},
        "engycapx": {"title": "UQx_ENGYCAPx", 'grade': 0.7},
    },
    'lgdm': {
        "lgdm1x": {"title": "UQx_LGDM1x", 'grade': 0.65},
        "lgdm2x": {"title": "UQx_LGDM2x",  'grade': 0.65},
        "lgdm3x": {"title": "UQx_LGDM3x",  'grade': 0.65},
        "lgdm4x": {"title": "UQx_LGDM4x",  'grade': 0.65},
        "lgdm5x": {"title": "UQx_LGDM5x",  'grade': 0.65},
    },
}

micromasters_courses = mmCoursesMeta[mmCourse]

def db_refresh():
    sqlite_connection.execute("DELETE FROM learners")
    sqlite_connection.execute("DELETE FROM course_instancedetails")
    sqlite_connection.execute("DELETE FROM " + mmCourse + "_coursecompletion")
    
def db_insert (table, row):
    keys = row.keys();
    sql = 'INSERT INTO ' + table + ' ('
    sql = sql + ', '.join(keys)
    sql = sql + ') VALUES ('
    sql = sql + ', '.join([ ('"' + str(row[key]) + '"') for key in keys])
    sql = sql + ')'
    id = sqlite_connection.execute(text(sql));
    return id
    
def import_profile_fromcsv(course_prefix):
    profile_file_to_import = grade_file_path + course_prefix + '_*' + profile_file_postfix
    #print(profile_file_to_import, course_prefix)
    
    for file_name in glob.glob(profile_file_to_import):
        with open(file_name, mode='r', encoding='utf-8-sig') as infile:
            reader = csv.DictReader(infile)
            for row in reader:
#                pprint(row)
                student_id = int(row['id'])
                email = row['email']
                email = email.replace("'", "")
                username = row['username']
                name = row['name']
#                if name is None:
#                    name = ''
                name = name.replace(':', '')
                name = name.replace('?', '')
                name = name.replace('\'', '')
                name = name.replace("\"", "")
                country = row['country']
                insert_learner(student_id, email, username, name,  country)
                
def insert_learner(student_id, email, username, name='',  country=''):
    sql = "SELECT COUNT(*) FROM learners where student_id=%d;" % (student_id)
    result = sqlite_connection.execute(sql);
    no_students = 0
    for row in result:
        no_students = row[0]
    if no_students == 0:
        row = {'student_id': student_id, 'email':email, 'username':username, 'name': name, 'country': country}
        db_insert('learners', row)
        
    
def import_gradereport_fromcsv(course, course_prefix):
    file_to_import = grade_file_path + course_prefix  + '_*' + grade_file_postfix
    #print(file_to_import, course, course_prefix)

    for file_name in glob.glob(file_to_import):
        course_instance_start = int(file_name.find(course_prefix))
        course_instance_end = file_name.find('_grade_report')
        course_instance = file_name[course_instance_start:course_instance_end]
        print(course_instance)
        with open(file_name, mode='r', encoding='utf-8-sig') as infile:
            reader = csv.DictReader(infile)
            for row in reader:
                student_id = int(row['Student ID'])
                email = row['Email']
                email = email.replace("'", "")
                username = row['Username']   
                insert_learner(student_id, email, username)
                #insert course instance details
                grade = row['Grade']
                enrollment_track = row['Enrollment Track']
                verification_status = row['Verification Status']
                certificate_eligible = row['Certificate Eligible']
                certificate_delivered = row['Certificate Delivered']
                row = {
                    'learner_id': student_id,
                    'grade': grade,
                    'enrollment_track': enrollment_track,
                    'verification_status': verification_status,
                    'certificate_eligible': certificate_eligible,
                    'certificate_delivered': certificate_delivered,
                    'course': course,
                    'course_instance': course_instance
                }
                db_insert('course_instancedetails', row)
        


def export_micromasters_completionreport():
    print("export_micromasters_completionreport")
    sql = "SELECT * FROM learners;"
    result = sqlite_connection.execute(sql);
    for row in result:
        learner_dict = {
            'student_id': row[1],
            'email': row[2],
            'name': row[3],
            'username': row[4],
            'country': row[5]
        }
        no_completed_courses = 0
        for course, course_dict in micromasters_courses.items():
            grade_certificate = course_dict['grade']
            learner_dict[course + "_" + 'verified'] = "-"
            learner_dict[course + "_" + 'completed'] = "-"
            courseinstance_sql = "SELECT * FROM course_instancedetails WHERE course='%s' AND learner_id=%d;" % (course, learner_dict['student_id'])
            courseinstance_result = sqlite_connection.execute(courseinstance_sql);
            verified_completed = False
            
            for courseinstance_row in courseinstance_result:
                enrollment_track = courseinstance_row[3]
                certificate_eligible = courseinstance_row[8]
                grade = courseinstance_row[2]
                
                grade_pass = 'N'
                if grade >= grade_certificate:
                    grade_pass = 'Y'
                if verified_completed == False:    
                    learner_dict[course + "_" + 'verified'] = enrollment_track
                    learner_dict[course + "_" + 'completed'] = grade_pass
                    if certificate_eligible == 'Y':
                        no_completed_courses += 1
                        verified_completed = True
                        break;
        
        learner_dict['no_completed_courses'] = no_completed_courses
        db_insert(mmCourse + '_coursecompletion', learner_dict)
                
def micromasters_completionreport_count_auditpassed():        
    print("micromasters_completionreport_count_auditpassed")
    sql = "SELECT * FROM " + mmCourse + "_coursecompletion;"
    result = sqlite_connection.execute(sql);
    for row in result:
        #pprint(row)
        update_id = int(row[0])
        no_auditedpassed_courses = 0
        if (row[5] == 'audit') and (row[6] == 'Y'):
            no_auditedpassed_courses +=1
        if (row[7] == 'audit') and (row[8] == 'Y'):
            no_auditedpassed_courses +=1
        if (row[9] == 'audit') and (row[10] == 'Y'):
            no_auditedpassed_courses +=1
        if (row[11] == 'audit') and (row[12] == 'Y'):
            no_auditedpassed_courses +=1
        
        updatesql = "UPDATE " + mmCourse + "_coursecompletion SET no_audited_passed=%d WHERE id=%d;" % (no_auditedpassed_courses, update_id)
        sqlite_connection.execute(updatesql);

        


        
db_refresh()
print('Import profile')
for course, course_dict in micromasters_courses.items():
    import_profile_fromcsv(course_dict['title'])

print('Import grade report')
for course, course_dict in micromasters_courses.items():
    import_gradereport_fromcsv(course, course_dict['title'])
    
export_micromasters_completionreport()
micromasters_completionreport_count_auditpassed()

print('The End.')



